In [1]:
import os
os.chdir('../')
%pwd

'/home/izam/coding/Sentiment-Analysis'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PredictionConfig:
    root_dir: str
    model_path: str
    vectorizer_path: str
    data_path: str
    prediction_file: str

In [3]:
from sentimentAnalysis.constants import *
from sentimentAnalysis.utils.common import read_yaml, create_directories, load_pickle, read_text

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_prediction_config(self) -> PredictionConfig:
        config = self.config.prediction

        create_directories([config.root_dir])

        prediction_config = PredictionConfig(
            root_dir = config.root_dir,
            model_path= config.model_path,
            vectorizer_path=config.vectorizer_path,
            data_path=config.data_path,
            prediction_file=config.prediction_file,
           
        )

        return prediction_config

In [5]:
import numpy as np
from sentimentAnalysis import logger
from sentimentAnalysis.utils.common import read_text, load_pickle, save_json
import json

In [6]:
class Prediction:
    def __init__(self, config: PredictionConfig):
        self.config = config

    def predict(self):
        model = load_pickle(Path(self.config.model_path))
        vectorizer = load_pickle(Path(self.config.vectorizer_path))
        data = read_text(path=Path(self.config.data_path))
        data = np.array(data)[:, 0]

        print(data)
        
        matrix = vectorizer.transform(data)
        prediction = model.predict(matrix)
        logger.info(f"predicted the new data as {prediction[0]}")


        save_json(path=self.config.prediction_file, data={'prediction':float(prediction[0])})
        
        

In [7]:
try:
    config = ConfigurationManager()
    prediction_config = config.get_prediction_config()
    prediction_config = Prediction(config=prediction_config)
    prediction_config.predict()
except Exception as e:
    raise e

[2023-12-10 00:40:13,962: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-10 00:40:13,964: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-10 00:40:13,965: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-10 00:40:13,966: INFO: common: created directory at: artifacts]
[2023-12-10 00:40:13,966: INFO: common: created directory at: artifacts/prediction]
[2023-12-10 00:40:14,207: INFO: common: pickle file loaded from: artifacts/model_trainer/model.pkl]
[2023-12-10 00:40:14,215: INFO: common: pickle file loaded from: artifacts/data_transformation/tfidf_vectorizer.pkl]
['you are bad']
[2023-12-10 00:40:14,217: INFO: 2887407288: predicted the new data as 0]
[2023-12-10 00:40:14,217: INFO: common: json file saved at: artifacts/prediction/prediction.json]
